In [13]:
import pandas as pd
pd.options.mode.chained_assignment = None
import altair as alt
alt.data_transformers.disable_max_rows()
import os
import sys
import warnings
warnings.filterwarnings('ignore')

In [2]:
user_df = pd.read_csv("../data/entity_files/users_dataset.csv")
repo_df = pd.read_csv("../data/large_files/entity_files/repos_dataset.csv", low_memory=False)
search_queries_repo_join_df = pd.read_csv("../data/join_files/search_queries_repo_join_dataset.csv")
search_queries_user_join_df = pd.read_csv("../data/join_files/search_queries_user_join_dataset.csv")

# core_repos = repo_df[repo_df["id"].isin(search_queries_repo_join_df["id"].unique())]
# core_users = user_df[(user_df.login.isin(top_contributors.login)) | (user_df.login.isin(search_queries_user_join_df.login)) | (user_df.login.isin(core_repos['owner.login']))].drop_duplicates(subset=['login'])

### Determine Core Users

In [33]:
core_dfs = []

#### Contributors

In [34]:
contributors_df = pd.read_csv('../data/join_files/repo_contributors_join_dataset.csv')
contributor_counts = contributors_df.groupby(['login']).size().reset_index(name='counts')
freq_value = contributor_counts.describe().loc['75%'].values[0]
top_contributors = contributor_counts[contributor_counts.counts > freq_value]
top_contributors['type'] = 'contributor'
core_dfs.append(top_contributors)
print(len(top_contributors))

573


In [10]:
alt.Chart(contributor_counts).mark_bar().encode(
    alt.X('counts', bin=alt.Bin(maxbins=100)),
    y='count()'
)

<VegaLite 4 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


#### Forks

In [35]:
forks_df = pd.read_csv('../data/join_files/repo_forks_join_dataset.csv')
forks_counts = forks_df.groupby(['owner.login']).size().reset_index(name='counts')
freq_value = forks_counts.describe().loc['75%'].values[0]
top_forks = forks_counts[forks_counts.counts > freq_value]
top_forks['type'] = 'fork'
top_forks = top_forks.rename(columns={'owner.login': 'login'})
core_dfs.append(top_forks)
print(len(top_forks))

168


In [9]:
alt.Chart(forks_counts).mark_bar().encode(
    alt.X('counts', bin=alt.Bin(maxbins=100)),
    y='count()'
)

<VegaLite 4 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


#### Stargazers

In [36]:
stargazers_df = pd.read_csv('../data/join_files/repo_stargazers_join_dataset.csv')
stargazers_counts = stargazers_df.groupby(['user.login']).size().reset_index(name='counts')
freq_value = stargazers_counts.describe().loc['75%'].values[0]
top_stargazers = stargazers_counts[stargazers_counts.counts > freq_value]
top_stargazers['type'] = 'stargazer'
top_stargazers = top_stargazers.rename(columns={'user.login': 'login'})
core_dfs.append(top_stargazers)
print(len(top_stargazers))

764


In [14]:
alt.Chart(stargazers_counts).mark_bar().encode(
    alt.X('counts', bin=alt.Bin(maxbins=100)),
    y='count()'
)

<VegaLite 4 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


#### Subscribers

In [37]:
subscribers_df = pd.read_csv('../data/join_files/repo_subscribers_join_dataset.csv')
subscribers_counts = subscribers_df.groupby(['login']).size().reset_index(name='counts')
freq_value = subscribers_counts.describe().loc['75%'].values[0]
top_subscribers = subscribers_counts[subscribers_counts.counts > freq_value]
top_subscribers['type'] = 'subscriber'
core_dfs.append(top_subscribers)
print(len(top_subscribers))

280


In [16]:
alt.Chart(subscribers_counts).mark_bar().encode(
    alt.X('counts', bin=alt.Bin(maxbins=100)),
    y='count()'
)

<VegaLite 4 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


#### Issues

In [41]:
issues_df = pd.read_csv('../data/large_files/join_files/repo_issues_join_dataset.csv')
issues_counts = issues_df.groupby(['user.login']).size().reset_index(name='counts')
freq_value = issues_counts.describe().loc['75%'].values[0]
top_issues = issues_counts[issues_counts.counts > freq_value]
top_issues['type'] = 'issue'
top_issues = top_issues.rename(columns={'user.login': 'login'})
core_dfs.append(top_issues)
print(len(top_issues))

775


In [20]:
alt.Chart(issues_counts).mark_bar().encode(
    alt.X('counts', bin=alt.Bin(maxbins=100)),
    y='count()'
)

<VegaLite 4 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


##### Issue Comments

In [24]:
issue_comments_df = pd.read_csv('../data/large_files/join_files/issues_comments_join_dataset.csv')
issue_comments_counts = issue_comments_df.groupby(['user.login']).size().reset_index(name='counts')
freq_value = issue_comments_counts.describe().loc['75%'].values[0]
top_issue_comments = issue_comments_counts[issue_comments_counts.counts > freq_value]
top_issue_comments['type'] = 'issue_comment'
top_issue_comments = top_issue_comments.rename(columns={'user.login': 'login'})
core_dfs.append(top_issue_comments)
print(len(top_issue_comments))

463


In [25]:
alt.Chart(issue_comments_counts).mark_bar().encode(
    alt.X('counts', bin=alt.Bin(maxbins=100)),
    y='count()'
)

<VegaLite 4 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


#### Pulls

In [26]:
pulls_df = pd.read_csv('../data/large_files/join_files/repo_pulls_join_dataset.csv')
pulls_counts = pulls_df.groupby(['user.login']).size().reset_index(name='counts')
freq_value = pulls_counts.describe().loc['75%'].values[0]
top_pulls = pulls_counts[pulls_counts.counts > freq_value]
top_pulls['type'] = 'pull'
top_pulls = top_pulls.rename(columns={'user.login': 'login'})
core_dfs.append(top_pulls)
print(len(top_pulls))

120


In [27]:
alt.Chart(pulls_counts).mark_bar().encode(
    alt.X('counts', bin=alt.Bin(maxbins=100)),
    y='count()'
)

<VegaLite 4 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


##### Pulls Comments

In [ ]:
pull_comments_df = pd.read_csv('../data/large_files/join_files/pulls_comments_join_dataset.csv')
pull_comments_counts = pull_comments_df.groupby(['user.login']).size().reset_index(name='counts')
freq_value = pull_comments_counts.describe().loc['75%'].values[0]
top_pull_comments = pull_comments_counts[pull_comments_counts.counts > freq_value]
top_pull_comments['type'] = 'pull_comment'
top_pull_comments = top_pull_comments.rename(columns={'user.login': 'login'})
core_dfs.append(top_pull_comments)
print(len(top_pull_comments))

In [ ]:
alt.Chart(pull_comments_counts).mark_bar().encode(
    alt.X('counts', bin=alt.Bin(maxbins=100)),
    y='count()'
)

### Explore Top Users

In [ ]:
top_users = pd.concat(core_dfs)

In [ ]:
grouped_counts_top_users = top_users.groupby(['login']).size().reset_index(name='unique_counts').sort_values(by=['unique_counts'], ascending=False)
grouped_sum_top_users = top_users.groupby(['login'])['counts'].sum().reset_index(name='sum_counts').sort_values(by=['sum_counts'], ascending=False)
merged_top_users = pd.merge(grouped_sum_top_users, grouped_counts_top_users, on='login')

In [ ]:
alt.Chart(merged_top_users).mark_circle().encode(
    alt.X('unique_counts'),
    alt.Y('sum_counts'),
    tooltip=['login', 'unique_counts', 'sum_counts']
)

In [ ]:
final_top_users = merged_top_users[merged_top_users.unique_counts > 1]